# Lab 3
+ ## Автор: Роман Кривохижа
+ ## Група: ІС-72
+ ## Викладач: Новікова П.А.

****
****
****

## Module importing

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set_style('darkgrid')
%matplotlib inline

## Algorithm implementation

+ Алгоритм, який відтворює функціонування системи, за допомогою комп’ютерної програми називається **алгоритмом імітації**.

+ Вимоги на обслуговування генеруються елементом **CREATE** і відправляються на обслуговування до елементу **PROCESS**, який здійснює обслуговування з часовою затримкою, заданою випадковим числом, та обмеження на довжину черги, заданим невід’ємним числом

<font size="4">Ймовірність відмови: $P = \frac{N_{unserv}}{N_{all}}$</font>

<font size="4">Середня довжина черги: $L_{aver} = \frac{\sum_{i} L_i \Delta t_i}{T_{mod}}$</font>

<font size="4">Середній час очікування: $Q_{aver} = \frac{\sum_{i} L_i \Delta t_i}{N_{serv}}$</font>

In [4]:
class Rand:
    """
    Генерація випадкового числа за заданим законом розподілу
    """
    @staticmethod
    def exp(time_mean):
        a = 0
        while a == 0:
            a = np.random.rand()
        return -time_mean * np.log(a)
    
    @staticmethod
    def unif(time_min, time_max):
        a = 0
        while a == 0:
            a = np.random.rand()
        a = time_min + a * (time+max - time_min)
        return a
    
    @staticmethod
    def norm(time_mean, time_deviation):
        return np.random.normal(loc=time_mean, scale=time_deviation)

In [84]:
class Element:
    id_curr = 0
    
    def __init__(self, name=None, delay_mean=1., delay_dev=0., distribution=''):
        self.tnext = 0.0
        self.delay_mean = delay_mean
        self.delay_dev = delay_dev
        self.distribution = distribution
        self.quantity = 0
        self.tcurr = self.tnext
        self.state = 0
        self.next_elements = None
        self.id_curr = Element.id_curr
        Element.id_curr += 1
        self.name = f'element_{self.id_curr}' if name is None else name
        
    def get_delay(self):
        if self.distribution == 'exp':
            return Rand.exp(self.delay_mean)
        elif self.distribution == 'unif':
            return Rand.unif(self.delay_mean, self.delay_dev)
        elif self.distribution == 'norm':
            return Rand.norm(self.delay_mean, self.delay_dev)
        else:
            return self.delay_mean
        
    def in_act(self):
        pass
    
    def out_act(self):
        self.quantity += 1
        
    def print_info(self):
        print(f'{self.name} state={self.state}; quantity={self.quantity}; tnext={self.tnext}')
        
    def print_result(self):
        print(f'{self.name} state={self.state}; quantity={self.quantity};')
    
    def do_statistics(self, delta):
        pass

In [85]:
class Create(Element):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def out_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        # встановлюємо коли пристрій буде вільним
        self.tnext = self.tcurr + super().get_delay()
        # передаємо створену заявку на настпуний елемент
        for next_element in self.next_elements:
            next_element.in_act()

In [86]:
class Process(Element):
    def __init__(self, maxqueue=np.inf, **kwargs):
        super().__init__(**kwargs)
        self.queue = 0
        self.maxqueue = maxqueue
        self.mean_queue_length = self.queue
        self.max_obs_queue_length = self.queue
        self.failure = 0
        
    def in_act(self):
        # перевіряємо чи вільний пристрій
        if self.state == 0:
            # позначаємо що пристрій зайнятий
            self.state = 1
            # встановлюємо коли пристрій буде вільним
            self.tnext = self.tcurr + super().get_delay()
        elif self.queue < self.maxqueue:
            self.queue += 1
            if self.queue > self.max_obs_queue_length:
                self.max_obs_queue_length = self.queue
        else:
            self.failure += 1
            
    def out_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        # позначаємо, що елемент вільний
        self.tnext = np.inf
        self.state = 0
        
        # дістаємо з черги елемент, якщо він там є
        if self.queue > 0:
            self.queue -= 1
            self.state = 1
            self.tnext = self.tcurr + super().get_delay()
        elif self.next_elements is not None:
            for next_element in self.next_elements:
                next_element.in_act()
        
    def print_info(self):
        super().print_info()
        print(f'failure={self.failure}\n')
        
    def do_statistics(self, delta):
        self.mean_queue_length =+ delta * self.queue

In [159]:
class Model():
    def __init__(self, elements=[], print_logs=False):
        self.elements = elements
        self.tnext = 0
        self.tcurr = self.tnext
        self.event = elements[0]
        self.print_logs = print_logs
        
    def simulate(self, time):
        self.max_time = time
        while self.tcurr < self.max_time:
            self.tnext = np.inf
            for e in self.elements:
                # знайти найменший з моментів часу
                if e.tnext < self.tnext:
                    self.tnext = e.tnext
                    self.event = e
            if self.print_logs:
                print(f'It`s time for event in {e.name}; time={self.tnext}')
            
            # обраховуємо статистики
            for e in self.elements:
                e.do_statistics(self.tnext - self.tcurr)
                
            # робимо переміщення до моменту завершення
            self.tcurr = self.tnext
            for e in self.elements:
                e.tcurr = self.tcurr
            
            # виконуємо операцію завершення
            self.event.out_act()
            
            # ????
            for e in self.elements:
                if e.tnext == self.tcurr:
                    e.out_act()
            if self.print_logs:        
                self.print_info()
        return self.print_result()
        
    def print_info(self):
        for e in self.elements:
            e.print_info()
            
    def print_result(self):
        print('-----RESULT-----')
        
        global_max_observed_queue_length = 0
        global_mean_queue_length_accumulator = 0
        global_failure_probability_accumulator = 0
        global_max_load = 0
        global_mean_load_accumulator = 0
        num_of_processors = 0
        
        for e in self.elements:
            e.print_result()
            if isinstance(e, Process):
                num_of_processors += 1
                mean_queue_length = e.mean_queue_length / self.tcurr
                failure_probability = e.failure / (e.quantity + e.failure)
                mean_load = e.quantity / self.max_time
                
                global_mean_queue_length_accumulator += mean_queue_length
                global_failure_probability_accumulator += failure_probability
                global_mean_load_accumulator += mean_load
                
                if e.max_obs_queue_length > global_max_observed_queue_length:
                    global_max_observed_queue_length = e.max_obs_queue_length
                    
                if mean_load > global_max_load:
                    global_max_load = mean_load
                
                if self.print_logs:
                    print(f"Average queue length: {mean_queue_length}")
                    print(f"Failure probability: {failure_probability}")
                    print(f"Average load: {mean_load}")
                
        global_mean_queue_length = global_mean_queue_length_accumulator / num_of_processors
        global_failure_probability = global_failure_probability_accumulator / num_of_processors
        global_mean_load = global_mean_load_accumulator / num_of_processors
        
        if self.print_logs:
            print(f"\nGlobal max observed queue length: {global_max_observed_queue_length}")
            print(f"Global mean queue length: {global_mean_queue_length}")
            print(f"Global failure probability: {global_failure_probability}")
            print(f"Global max load: {global_max_load}")
            print(f"Global mean load: {global_mean_load}")
        
        return {
            "global_max_observed_queue_length": global_max_observed_queue_length,
            "global_mean_queue_length": global_mean_queue_length,
            "global_failure_probability": global_failure_probability,
            "global_max_load": global_max_load,
            "global_mean_load": global_mean_load
        }

In [160]:
class SimModel():
    def __init__(self):
        c1 = Create(delay_mean=2.0, name='CREATOR_1', distribution='exp')
        p1 = Process(maxqueue=2, delay_mean=4.0, name='PROCESSOR1', distribution='exp')
#         p2 = Process(maxqueue=5, delay_mean=1.0, name='PROCESSOR2', distribution='exp')
#         p3 = Process(maxqueue=5, delay_mean=1.0, name='PROCESSOR3', distribution='exp')
#         p4 = Process(maxqueue=5, delay_mean=1.0, name='PROCESSOR4', distribution='exp')
#         p5 = Process(maxqueue=5, delay_mean=1.0, name='PROCESSOR5', distribution='exp')
        
#         c1.next_elements = [p1]
#         p1.next_elements = [p2, p3]
#         p3.next_elements = [p4]
#         p4.next_elements = [p1]
        
#         elements = [c1, p1, p2, p3, p4]

        c1.next_elements = [p1]
        elements = [c1,p1]
        model = Model(elements, print_logs=True)
        model.simulate(10)

In [161]:
Element.id_curr = 0
s = SimModel()

It`s time for event in PROCESSOR1; time=0.0
CREATOR_1 state=0; quantity=1; tnext=7.33756831162432
PROCESSOR1 state=1; quantity=0; tnext=0.23757028679259762
failure=0

It`s time for event in PROCESSOR1; time=0.23757028679259762
CREATOR_1 state=0; quantity=1; tnext=7.33756831162432
PROCESSOR1 state=0; quantity=1; tnext=inf
failure=0

It`s time for event in PROCESSOR1; time=7.33756831162432
CREATOR_1 state=0; quantity=2; tnext=9.478172317869303
PROCESSOR1 state=1; quantity=1; tnext=8.614390079542614
failure=0

It`s time for event in PROCESSOR1; time=8.614390079542614
CREATOR_1 state=0; quantity=2; tnext=9.478172317869303
PROCESSOR1 state=0; quantity=2; tnext=inf
failure=0

It`s time for event in PROCESSOR1; time=9.478172317869303
CREATOR_1 state=0; quantity=3; tnext=9.762854125990293
PROCESSOR1 state=1; quantity=2; tnext=10.492849154103567
failure=0

It`s time for event in PROCESSOR1; time=9.762854125990293
CREATOR_1 state=0; quantity=4; tnext=10.823545598569208
PROCESSOR1 state=1; quanti

**Проведемо верифікацію, використовуючи побудовану таблицю:**

In [165]:
n_param = 10

delay_create_list = list(range(1, n_param+1))
delay_process_list = list(range(1, n_param+1))
maxQ_list = list(range(1, n_param+1))

time_modeling_list = [i*100 for i in range(1, n_param+1)]

np.random.shuffle(delay_create_list)
np.random.shuffle(delay_process_list)
np.random.shuffle(maxQ_list)
np.random.shuffle(time_modeling_list)

df = pd.DataFrame()
rows = []

for i in range(n_param):
    c1 = Create(delay_mean=delay_create_list[i], name='CREATOR_1', distribution='exp')
    p1 = Process(maxqueue=maxQ_list[i], delay_mean=delay_process_list[i], name='PROCESSOR1', distribution='exp')

    c1.next_elements = [p1]
    elements = [c1, p1]
    model = Model(elements, print_logs=False)
    res = model.simulate(time_modeling_list[i])
    
    param = {'delay_mean_create': delay_create_list[i], 'delay_process_create': delay_process_list[i],
             'distribution': 'exp', 'maxqueue': maxQ_list[i]}
    
    rows.append({**param, **res})
df = df.append(rows)

-----RESULT-----
CREATOR_1 state=0; quantity=127;
PROCESSOR1 state=1; quantity=89;
-----RESULT-----
CREATOR_1 state=0; quantity=98;
PROCESSOR1 state=1; quantity=21;
-----RESULT-----
CREATOR_1 state=0; quantity=34;
PROCESSOR1 state=1; quantity=28;
-----RESULT-----
CREATOR_1 state=0; quantity=99;
PROCESSOR1 state=1; quantity=45;
-----RESULT-----
CREATOR_1 state=0; quantity=20;
PROCESSOR1 state=1; quantity=16;
-----RESULT-----
CREATOR_1 state=0; quantity=123;
PROCESSOR1 state=1; quantity=122;
-----RESULT-----
CREATOR_1 state=0; quantity=986;
PROCESSOR1 state=1; quantity=337;
-----RESULT-----
CREATOR_1 state=0; quantity=129;
PROCESSOR1 state=1; quantity=96;
-----RESULT-----
CREATOR_1 state=0; quantity=47;
PROCESSOR1 state=1; quantity=46;
-----RESULT-----
CREATOR_1 state=0; quantity=267;
PROCESSOR1 state=1; quantity=196;


In [166]:
df

,delay_mean_create,delay_process_create,distribution,maxqueue,global_max_observed_queue_length,global_mean_queue_length,global_failure_probability,global_max_load,global_mean_load
0,7,9,exp,4,4,0.021052,0.270492,0.098889,0.098889
1,2,7,exp,10,10,0.069468,0.758621,0.105000,0.105000
2,9,8,exp,6,6,0.020715,0.096774,0.093333,0.093333
3,4,10,exp,7,7,0.031785,0.505495,0.112500,0.112500
4,8,2,exp,1,1,0.029606,0.157895,0.160000,0.160000
5,6,1,exp,9,4,0.000000,0.000000,0.174286,0.174286
6,1,3,exp,2,2,0.001747,0.657172,0.337000,0.337000
7,5,6,exp,3,3,0.023631,0.238095,0.160000,0.160000
8,10,5,exp,8,4,0.000000,0.000000,0.092000,0.092000
9,3,4,exp,5,5,0.048267,0.249042,0.245000,0.245000


**Проведемо верифікацію, використовуючи побудовану таблицю:**

## Conclusion

**В данній лабораторній роботі ми реалізували алгоритм імітації простої моделі обслуговування, використовуючи спосіб, який орієнтований на події (аби уникнути випадку, коли дві події приходять в один час).**
+ за допомогою статичних методів була зібрана інформація про роботу/поведінку моделі
+ зміна віхдних параметрів моделі призводить до зміни вихідних значень
+ при досить великій кількості вхідинх параметрів важко проводити етап верифікації моделі

P.S. нижче наведено декілька графіків, які ілюструють залежність вихідних параметрів від вхідних